# 🏋️ AI Search + Agent Service: Fitness-Fun Example 🤸

Welcome to our **AI Search** + **AI Agent** tutorial, where we'll:

- **Create** an Azure AI Search index with fitness-oriented sample data
- **Demonstrate** search functionality with fitness equipment queries
- **Create** an AI agent with fitness knowledge and health disclaimers
- **Show** how to have conversations about fitness advice and equipment recommendations

## 🏥 Health & Fitness Disclaimer
>   **This notebook is for general demonstration and entertainment purposes, NOT a substitute for professional medical advice.** Always seek the advice of certified health professionals.   

## Prerequisites

- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- An **Azure AI Search** resource (formerly "Cognitive Search"), provisioned in your Azure AI Foundry project.

## What You'll Learn
- ✅ Azure AI Search: Create indexes, upload documents, perform searches
- ✅ Agent Service: Create AI agents with domain-specific knowledge
- ✅ Integration Patterns: How to connect search functionality with AI agents
- ✅ Best Practices: Error handling, resource cleanup, health disclaimers

## High-Level Flow

We'll do the following:

1. **Create** an AI Search index programmatically with sample fitness data.
2. **Upload** documents (fitness items) to the index.
3. **Verify** search functionality with test queries.
4. **Create** an AI agent with fitness expertise and proper disclaimers.
5. **Test** agent conversations about fitness equipment and advice.
6. **Clean up** resources responsibly.

<img src="./seq-diagrams/5-ai-search.png" alt="AI Search + Agent Service Flow" width="800"/>

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:

- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 📋 Prerequisites: Create Azure AI Search Connection

Before proceeding with this tutorial, you need to create an Azure AI Search connection in your AI Foundry project.

**Steps to Create the Connection:**
1. **Navigate to Azure AI Foundry portal:** https://ai.azure.com/
2. **Go to Foundry → Operate → Admin → Your Project**
3. **Click "Add Connection" → Azure AI Search**
4. **Provide the following details:**
   - **Connection Name**: Give it a descriptive name (e.g., "my-ai-search")
   - **Subscription**: Select your Azure subscription
   - **Resource Group**: Choose your resource group
   - **Azure AI Search Resource**: Select your existing search service or create a new one
5. **Click "Connect"** to complete the setup

## Alternative: Use Azure CLI

**Create AI Search connection via CLI**
```
az ml connection create --file connection.yaml --resource-group <your-rg> --workspace-name <your-project>
```

## ⚠️ Important Notes:
- The connection must be created before running the code below
- The code will automatically retrieve the default AI Search connection from your project
- If you don't have an Azure AI Search resource, you'll need to create one first

## 1. Create & Populate Azure AI Search Index with Vector Search

We'll create a **vector search enabled** index called `myfitnessindex`, containing fitness equipment with both traditional fields and vector embeddings.

Key Features:

- **Vector Field**: DescriptionVector stores embeddings for semantic search
- **HNSW Algorithm**: Fast approximate nearest neighbor search
- **Azure OpenAI Vectorizer**: Integrated embedding generation
- **Semantic Understanding**: Searches based on meaning, not just keywords

This enables the agent to understand queries like "affordable equipment" or "cheapest items" even when those exact words aren't in the product descriptions!

In [2]:
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: Azure.Search.Documents, 11.7.0"
#r "nuget: dotenv.net"

using System.IO;
using System.ClientModel.Primitives;
using System.Dynamic;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI;
using OpenAI.Embeddings;
using OpenAI.Responses;
using OpenAI.Chat;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents.Models;
using dotenv.net;

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

Installed Packages Azure.AI.Projects, 1.2.0-beta.5 Azure.Identity, 1.18.0-beta.2 Azure.Search.Documents, 11.7.0 dotenv.net, 4.0.0

In [3]:
#pragma warning disable OPENAI001

var aiFoundryProjectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var chatModel = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
var textEmbeddingModel = Environment.GetEnvironmentVariable("TEXT_EMBEDDING_MODEL");
var textEmbeddingDimension = 1536; // e.g., 1536 for ada-002
if(textEmbeddingModel == "text-embedding-3-large")
{
    textEmbeddingDimension = 3072;
}
var searchIndexName = "myfitnessindex";
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");

AIProjectClient projectClient;
OpenAIClient openAIClient;
EmbeddingClient embeddingClient;

string openAiEndpoint;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    // Validate credentials by getting a token
    var tokenRequestContext = new TokenRequestContext(new[] { "https://ai.azure.com/.default" });
    var accessToken = await credential.GetTokenAsync(tokenRequestContext);
    Console.WriteLine("✅ Successfully initialized Azure credentials with correct tenant!");      

    projectClient = new(new Uri(aiFoundryProjectEndpoint), credential);
    Console.WriteLine("🎉 Successfully created AIProjectClient");

    openAiEndpoint = $"https://{projectClient.OpenAI.Endpoint.Authority.Replace(".services.ai.",".openai.")}/openai/v1";
    Console.WriteLine($"Using OpenAI endpoint: {openAiEndpoint}");
    BearerTokenPolicy tokenPolicy = new(
        credential,
        "https://ai.azure.com/.default");

    openAIClient = new(
        authenticationPolicy: tokenPolicy,
        options: new OpenAIClientOptions()
        {
            Endpoint = new(openAiEndpoint),
        }
    );

    embeddingClient = openAIClient.GetEmbeddingClient(textEmbeddingModel);

    Console.WriteLine("🎉 Successfully created OpenAIClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

✅ Successfully initialized Azure credentials with correct tenant!
🎉 Successfully created AIProjectClient
Using OpenAI endpoint: https://aifp-ai2-dev-resource.openai.azure.com/openai/v1
🎉 Successfully created OpenAIClient


In [4]:
var searchConn = await projectClient.Connections.GetDefaultConnectionAsync(connectionType: ConnectionType.AzureAISearch, includeCredentials: true);
string searchEndPoint = searchConn.Target;
var searchApiKey = (searchConn.Credentials as AIProjectConnectionApiKeyCredential)?.ApiKey;
var credential = new AzureKeyCredential(searchApiKey);
var searchIndexClient = new SearchIndexClient(new Uri(searchEndPoint), credential);
var searchClient = new SearchClient(new Uri(searchEndPoint), searchIndexName, credential);

Console.WriteLine($"✅ Initialized Search clients for index: {searchIndexName}");

✅ Initialized Search clients for index: myfitnessindex


In [5]:
var fields = new List<SearchField>
{
    new SimpleField("FitnessItemID", SearchFieldDataType.String) { IsKey = true },
    new SearchableField("Name") { IsFilterable = true },
    new SearchableField("Category") { IsFilterable = true, IsFacetable = true },
    new SimpleField("Price", SearchFieldDataType.Double) { IsFilterable = true, IsSortable = true, IsFacetable = true },
    new SearchField("Description", SearchFieldDataType.String),
    new SearchField("DescriptionVector", SearchFieldDataType.Collection(SearchFieldDataType.Single))
    {
        IsSearchable = true,
        VectorSearchDimensions = textEmbeddingDimension,
        VectorSearchProfileName = "myHnswProfile"
    }
};

var vectorizer = new AzureOpenAIVectorizer("myOpenAI")
{
    Parameters = new AzureOpenAIVectorizerParameters
    {
        ResourceUri = new Uri(openAiEndpoint),
        DeploymentName = textEmbeddingModel,
        ModelName = textEmbeddingModel
    }
};

VectorSearch vectorSearch = new() 
{
        Profiles = { new VectorSearchProfile("myHnswProfile", "myHnsw") },
        Algorithms = { new HnswAlgorithmConfiguration("myHnsw")  },
        Vectorizers = { vectorizer }
};

SemanticConfiguration semanticConfig = new("my-semantic-config", new(){
        TitleField = new SemanticField("Name"),
        ContentFields = { new SemanticField("Description") },
        KeywordsFields = { new SemanticField("Category") }
});

SemanticSearch semanticSearch = new()
{
    Configurations = { semanticConfig }
};

var indexes = searchIndexClient.GetIndexNamesAsync();
await foreach(var index in indexes)
{
    if(index == searchIndexName)
    {
        await searchIndexClient.DeleteIndexAsync(searchIndexName);
        break;
    }
}

var indexDefinition = new SearchIndex(searchIndexName)
{
    Fields = fields,
    VectorSearch = vectorSearch,
    SemanticSearch = semanticSearch
};

var searchIndex = await searchIndexClient.CreateIndexAsync(indexDefinition);

Console.WriteLine($"🎉 Created vector search index with semantic configuration: {searchIndex.Value.Name}");
Console.WriteLine($"   Using embedding model: {textEmbeddingModel}");
Console.WriteLine($"   Vector dimensions: {textEmbeddingDimension}");
Console.WriteLine($"   Semantic configuration: my-semantic-config");

🎉 Created vector search index with semantic configuration: myfitnessindex
   Using embedding model: text-embedding-3-small
   Vector dimensions: 1536
   Semantic configuration: my-semantic-config


In [6]:
async Task<float[]> GenerateEmbedding(string text)
{
    var response = await embeddingClient.GenerateEmbeddingAsync(text);
    return response.Value.ToFloats().ToArray();
}

var sampleDocs = new List<Dictionary<string, object>>()
{
    new() {
        ["FitnessItemID"] = "1",
        ["Name"] = "Adjustable Dumbbell",
        ["Category"] = "Strength", 
        ["Price"] = 59.99,
        ["Description"] = "A compact, adjustable weight for targeted muscle workouts. Perfect for home gym setups with adjustable weight settings from 5 to 50 pounds.",
        ["DescriptionVector"] = (float[])null
    },
    new() {
        ["FitnessItemID"] = "2",
        ["Name"] = "Yoga Mat",
        ["Category"] = "Flexibility",
        ["Price"] = 25.0,
        ["Description"] = "Non-slip mat designed for yoga, Pilates, and other exercises. Extra thick padding for comfort during floor exercises.",
        ["DescriptionVector"] = (float[])null
    },
    new() {
        ["FitnessItemID"] = "3",
        ["Name"] = "Treadmill",
        ["Category"] = "Cardio",
        ["Price"] = 499.0,
        ["Description"] = "A sturdy treadmill with adjustable speed and incline settings. Features digital display, heart rate monitor, and multiple workout programs for cardio fitness.",
        ["DescriptionVector"] = (float[])null
    },
    new() {
        ["FitnessItemID"] = "4",
        ["Name"] = "Resistance Bands",
        ["Category"] = "Strength",
        ["Price"] = 15.0,
        ["Description"] = "Set of colorful bands for light to moderate resistance workouts. Includes five different resistance levels for progressive strength training.",
        ["DescriptionVector"] = (float[])null
    },
    new() {
        ["FitnessItemID"] = "5",
        ["Name"] = "Foam Roller",
        ["Category"] = "Recovery",
        ["Price"] = 29.99,
        ["Description"] = "High-density foam roller for muscle recovery and myofascial release. Helps reduce muscle soreness and improves flexibility after workouts.",
        ["DescriptionVector"] = (float[])null
    },
    new() {
        ["FitnessItemID"] = "6",
        ["Name"] = "Kettlebell Set",
        ["Category"] = "Strength",
        ["Price"] = 89.99,
        ["Description"] = "Professional kettlebell set with multiple weights ranging from 10 to 35 pounds. Ideal for full-body strength and conditioning workouts.",
        ["DescriptionVector"] = (float[])null
    }
};

Console.WriteLine("Generating embeddings for documents...");
foreach(var doc in sampleDocs)
{
    var textForEmbedding = $"{doc["Name"]} {doc["Category"]} {doc["Description"]} Price:{doc["Price"]}";
    var embedding = await GenerateEmbedding(textForEmbedding);
    doc["DescriptionVector"] = embedding;
    Console.WriteLine($"  ✓ Generated embedding for: {doc["Name"]}");
}

var result = await searchClient.UploadDocumentsAsync(sampleDocs);
Console.WriteLine($"🚀 Uploaded {sampleDocs.Count} documents with embeddings to index");
Console.WriteLine(" Documents now support semantic vector search!");

Generating embeddings for documents...
  ✓ Generated embedding for: Adjustable Dumbbell
  ✓ Generated embedding for: Yoga Mat
  ✓ Generated embedding for: Treadmill
  ✓ Generated embedding for: Resistance Bands
  ✓ Generated embedding for: Foam Roller
  ✓ Generated embedding for: Kettlebell Set
🚀 Uploaded 6 documents with embeddings to index
 Documents now support semantic vector search!


In [7]:
System.Threading.Thread.Sleep(2000); // wait for indexing to complete
string[] testQueries = {
    "equipment for building strength at home",
    "tools for muscle recovery and flexibility",
    "cardio training equipment for fitness"
};

foreach(var query in testQueries)
{
    Console.WriteLine($"🔍 Vector Search: '{query}'");
    Console.WriteLine("".PadRight(60, '-'));

    var queryEmbedding = await GenerateEmbedding(query);

    var vectorQuery = new VectorizedQuery(queryEmbedding){
        KNearestNeighborsCount = 3,
        Fields = { "DescriptionVector" }
    };

    var results = await searchClient.SearchAsync<Dictionary<string, object>>(searchText: null,
        new SearchOptions()
        {
            VectorSearch = new VectorSearchOptions(){
                Queries = { vectorQuery }
            },
            Select = { "Name", "Category", "Price", "Description" }
        });

    await foreach (var resultItem in results.Value.GetResultsAsync())
    {
        var doc = resultItem.Document;
        Console.WriteLine($"✓ {doc["Name"]} - ${doc["Price"]:.2f}");
        Console.WriteLine($"  Category: {doc["Category"]}");
        Console.WriteLine($"  Description: {doc["Description"].ToString().Substring(0, 80)}...");
    }
}

Console.WriteLine("\n ✅ Vector search is working! The index finds items based on semantic meaning.");
Console.WriteLine("   These queries focus on concepts, categories, and use cases - ideal for vector search!");

🔍 Vector Search: 'equipment for building strength at home'
------------------------------------------------------------
✓ Adjustable Dumbbell - $59.99
  Category: Strength
  Description: A compact, adjustable weight for targeted muscle workouts. Perfect for home gym ...
✓ Resistance Bands - $15.0
  Category: Strength
  Description: Set of colorful bands for light to moderate resistance workouts. Includes five d...
✓ Kettlebell Set - $89.99
  Category: Strength
  Description: Professional kettlebell set with multiple weights ranging from 10 to 35 pounds. ...
🔍 Vector Search: 'tools for muscle recovery and flexibility'
------------------------------------------------------------
✓ Foam Roller - $29.99
  Category: Recovery
  Description: High-density foam roller for muscle recovery and myofascial release. Helps reduc...
✓ Yoga Mat - $25.0
  Category: Flexibility
  Description: Non-slip mat designed for yoga, Pilates, and other exercises. Extra thick paddin...
✓ Resistance Bands - $15.0
  

## 2. Create Agent With Azure AI Search Tool

Configure the official AzureAISearchTool and create an AI agent with direct search integration.

In [9]:
AzureAISearchToolIndex aiSearchToolIndex = new()
{
    ProjectConnectionId = searchConn.Id,
    IndexName = searchIndexName,
    QueryType = AzureAISearchQueryType.Semantic,
    TopK = 5,
};
PromptAgentDefinition agentDefinition = new(model: chatModel)
{
    Instructions = @"
    You are a Fitness Shopping Assistant with access to a live product catalog through Azure AI Search with SEMANTIC SEARCH capabilities powered by vector embeddings.

    **Your Capabilities:**
    - Use semantic search to find products based on meaning and context, not just keywords
    - Search by price range, features, categories, and benefits
    - Understand natural language queries like ""affordable"", ""budget-friendly"", ""cheapest""
    - The index uses vector embeddings, so you can find items based on semantic similarity
    - Provide fitness advice with proper health disclaimers
    - Recommend equipment based on goals, budget, and safety

    **Search Strategy:**
    - When users ask about prices (e.g., ""under $100"", ""cheapest"", ""affordable""), search the catalog
    - The semantic search with vector embeddings understands meaning - you can find items based on descriptions
    - Always include specific prices and product details in your responses
    - Compare prices when relevant to help users make informed decisions

    **Guidelines:**
    - Always search for products when users ask about specific categories, prices, or items
    - Include safety considerations and proper usage tips
    - Remind users this is educational only - consult healthcare providers for medical advice
    - Be specific about prices and features when recommending products

    Be helpful, encouraging, and prioritize user safety while leveraging the powerful semantic search to find exactly what users need!
    ",
    Tools = { new AzureAISearchAgentTool(new AzureAISearchToolOptions(indexes: [aiSearchToolIndex])) }
};
AgentVersion semanticSearchAgent = await projectClient.Agents.CreateAgentVersionAsync(
    agentName: "fitness-semantic-search-agent",
    options: new(agentDefinition)
);

Console.WriteLine($"🎉 Created agent: {semanticSearchAgent.Name} (ID: {semanticSearchAgent.Id})");
Console.WriteLine($"✅ Connected to vector-enabled search index: {searchIndexName}");
Console.WriteLine($"🔍 Using SEMANTIC query type (powered by vector embeddings)");

🎉 Created agent: fitness-semantic-search-agent (ID: fitness-semantic-search-agent:1)
✅ Connected to vector-enabled search index: myfitnessindex
🔍 Using SEMANTIC query type (powered by vector embeddings)


# 3. Test Agent with Search Integration

Test the agent with fitness equipment questions. The agent will automatically use the Azure AI Search tool when needed.

In [11]:
#pragma warning disable OPENAI001

async Task TestAgentQuestionAsync(string question)
{
    Console.WriteLine($"\n🔍 **Question:** {question}");
    Console.WriteLine(new string('=', 70));

    var conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();

    var responseClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(semanticSearchAgent, conversation.Value.Id);
    var response = await responseClient.CreateResponseAsync(question);
    if(response.Value.Status == ResponseStatus.Completed)
    {
        Console.WriteLine("🤖 **Agent Response:**");
        Console.WriteLine(response.Value.GetOutputText());
    }
    else
    {
        Console.WriteLine($"❌ Agent response failed with status: {response.Value.Status}");
    }
    Console.WriteLine(new string('=', 70));
}

string[] testQuestions = [
    "What affordable strength training equipment do you have? I'm looking to spend under $100",
    "I need equipment for muscle recovery and reducing soreness after workouts",
    "What cardio equipment options do you have? Please include prices",
    "Show me budget-friendly strength training options for a home gym",
    "I'm interested in flexibility and stretching equipment. What do you recommend?",
    "What's good for full-body conditioning workouts at different price points?"
];

Console.WriteLine("🧪 Testing Agent with Semantic Search - Context & Concept Queries");
Console.WriteLine(new string('=', 70));
Console.WriteLine("Note: These queries work well with semantic search as they focus on");
Console.WriteLine("      concepts, categories, and general needs rather than exact filters.\n");

foreach(var question in testQuestions)
{
    await TestAgentQuestionAsync(question);
}

Console.WriteLine("✅ Agent testing completed with vector search + semantic configuration!");
Console.WriteLine("   Semantic search excels at understanding context, categories, and natural language.");
Console.WriteLine("   For exact price filtering, consider combining with OData filters (e.g., 'Price le 50').");

🧪 Testing Agent with Semantic Search - Context & Concept Queries
Note: These queries work well with semantic search as they focus on
      concepts, categories, and general needs rather than exact filters.


🔍 **Question:** What affordable strength training equipment do you have? I'm looking to spend under $100
🤖 **Agent Response:**
I found a few affordable strength training equipment options under $100:

1. Resistance Bands - These come in a set with five different resistance levels, great for light to moderate strength training and progressive workouts.

2. Kettlebell Set - Professional kettlebell set ranging from 10 to 35 pounds for full-body strength and conditioning.

3. Adjustable Dumbbell - Compact, adjustable dumbbell with weights adjustable from 5 to 50 pounds, suitable for targeted muscle workouts at home.

Would you like specific prices and more details on any of these items? Or help choosing the best option for your strength goals and budget?

🔍 **Question:** I need equipme

In [12]:
// Clean up resources
await projectClient.Agents.DeleteAgentAsync(semanticSearchAgent.Name);
await searchIndexClient.DeleteIndexAsync(searchIndexName);

Console.WriteLine("🗑️ Deleted agent and search index");
Console.WriteLine("✅ Cleanup completed!");

🗑️ Deleted agent and search index
✅ Cleanup completed!


## 🎉 Congratulations!

You've successfully completed the AI Search + Agent Service with Vector Search tutorial!

### ✅ What We Accomplished

1. 🔍 **Azure AI Search with Vector Search & Semantic Configuration**

- Created a search index with **vector embeddings** for semantic search
- Configured **HNSW algorithm** for efficient vector similarity search
- Used **Azure OpenAI embeddings** (text-embedding-ada-002) to vectorize product descriptions
- Implemented **integrated vectorization** with AzureOpenAIVectorizer
- Added **Semantic Configuration** (required for SEMANTIC query type)
- Included price information in embeddings for better context understanding

2. 🤖 **AI Agent with Semantic Search Capabilities**

- Created an agent using **SEMANTIC query type** (powered by vector embeddings)
- Agent understands semantic meaning and can find products based on:
  - **Categories** ("strength training", "cardio", "flexibility")
  - **Use cases** ("muscle recovery", "home gym", "conditioning")
  - **Concepts** ("affordable", "budget-friendly", "full-body")
  - **Natural language** queries without exact keyword matches
- Seamless integration with real-time semantic search results

3. 💡 Key Improvements Over Simple Search

- **Semantic Understanding**: Finds relevant items based on meaning, not just keywords
- **Context Awareness**: Understands "muscle recovery" → foam roller, yoga mat
- **Better Relevance**: Returns results based on semantic similarity
- **Richer Embeddings**: Combined name, category, description, and price in vector embeddings
- **Natural Language**: Understands intent from conversational queries

4. 📊 **Technical Implementation**

- **Vector Field**: DescriptionVector with 1536 dimensions
- **Vector Search**: HNSW algorithm for fast approximate nearest neighbor search
- **Semantic Config**: Required for SEMANTIC query type with Name, Description, Category fields
- **Manual Vectorization**: Pre-computed embeddings using Azure OpenAI
- **Official SDK**: Uses azure-search-documents and azure-ai-agents packages

## 🎯 How Vector Search Works in This Example

1. **Indexing Time:**

- Documents are enriched with text combining all relevant fields + price
- Azure OpenAI generates 1536-dimensional embeddings
- Embeddings stored in DescriptionVector field
- HNSW algorithm creates efficient search structure

2. **Query Time:**

- User asks a natural language question to the agent
- Agent uses SEMANTIC search (leverages pre-computed vectors)
- Search returns most semantically similar items
- Agent formats results with prices and details

## 🔧 Technical Architecture
User Query (Natural Language) <br>
    ↓<br>
AI Agent (SEMANTIC query type)<br>
    ↓<br>
Azure AI Search Index<br>
    ├─ Traditional fields (Name, Price, Category)<br>
    └─ Vector field (DescriptionVector with embeddings)<br>
    ↓<br>
HNSW Algorithm (similarity search)<br>
    ↓<br>
Semantic Results (most similar items)<br>
    ↓<br>
Agent Response (formatted with context)<br>

## 📖 Key Learnings

1. **Vector Search Types:**

- **VECTOR query type**: Requires runtime vectorization (authentication challenges)
- **SEMANTIC query type**: Uses pre-computed vectors (what we implemented)
- Both leverage the same vector index and embeddings

2. **Semantic Configuration:**

- Required for SEMANTIC query type
- Defines title, content, and keyword fields
- Improves relevance ranking

3. **Embedding Strategy:**

- Include all relevant context in text for embedding
- Price information embedded as text: "Price: $59.99"
- Enables semantic understanding of price concepts

4. **Best Query Patterns for Semantic Search:**

- ✅ Good: "affordable strength training equipment" (concepts)
- ✅ Good: "equipment for muscle recovery" (use cases)
- ✅ Good: "budget-friendly home gym options" (categories)
- ❌ Limited: "show items under $50" (exact numerical filters)
- ❌ Limited: "what's the cheapest item" (superlatives requiring sorting)

5. **When to Use What:**

- **Semantic Search**: Best for understanding intent, categories, concepts
- **Filters**: Best for exact numerical ranges (combine with OData: Price le 100)
- **Hybrid**: Combine semantic search + filters for best results

## 🚀 Next Steps & Improvements

1. **Add OData Filters**: Combine semantic search with exact price filters

    `filter="Price le 100"  # Combine with semantic search`

2. **Hybrid Search**: Combine keyword search + vector search

    `search_text="strength",  # Keyword search` <br>
    `vector_queries=[vector_query]  # Vector search`

3. **Semantic Ranker**: Enable L2 semantic ranking for better relevance

4. **Multi-field Vectors**: Create separate vectors for different aspects

5. **Query Expansion**: Let agent rephrase queries for better results

## 🎓 Summary

**Semantic Search Excels At:**

- Understanding natural language and intent
- Finding items by category, use case, or concept
- Providing contextually relevant results
- Handling synonyms and related terms

**For Exact Filtering, Add:**

- OData filters for price ranges
- Traditional field filters
- Faceting for categorical filtering

## 📚 Resources
- [Vector Search Overview](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)
- [Create Vector Index](https://learn.microsoft.com/en-us/azure/search/search-create-vector-index)
- [Vector Query Guide](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-query)
- [Azure Search Vector Samples](https://github.com/Azure/azure-search-vector-samples)
- [OData Filters in Azure Search](https://learn.microsoft.com/en-us/azure/search/search-query-odata-filter)

Ready to build production AI applications with Azure AI Search Vector Search + Agent integration! 🚀